In [ ]:
# h5py 안 될 때
!brew reinstall hdf5
!export CPATH="/opt/homebrew/include/"
!export HDF5_DIR=/opt/homebrew/
!python3 -m pip install h5py

In [210]:
import pickle
import math
from copy import deepcopy

import h5py
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn

In [211]:
path_root = '../'
path_container = './Container/'

### Dataset 가져오기 (1) dict

In [212]:
with open(path_container + 'id_cuisine_dict.pickle', 'rb') as f:
    id_cuisine_dict = pickle.load(f)

print(id_cuisine_dict)
type(id_cuisine_dict), len(id_cuisine_dict)

{0: 'brazilian', 1: 'british', 2: 'cajun_creole', 3: 'chinese', 4: 'filipino', 5: 'french', 6: 'greek', 7: 'indian', 8: 'irish', 9: 'italian', 10: 'jamaican', 11: 'japanese', 12: 'korean', 13: 'mexican', 14: 'moroccan', 15: 'russian', 16: 'southern_us', 17: 'spanish', 18: 'thai', 19: 'vietnamese'}


(dict, 20)

In [213]:
with open(path_container + 'cuisine_id_dict.pickle', 'rb') as f:
    cuisine_id_dict = pickle.load(f)

print(cuisine_id_dict)
type(cuisine_id_dict), len(cuisine_id_dict)

{'brazilian': 0, 'british': 1, 'cajun_creole': 2, 'chinese': 3, 'filipino': 4, 'french': 5, 'greek': 6, 'indian': 7, 'irish': 8, 'italian': 9, 'jamaican': 10, 'japanese': 11, 'korean': 12, 'mexican': 13, 'moroccan': 14, 'russian': 15, 'southern_us': 16, 'spanish': 17, 'thai': 18, 'vietnamese': 19}


(dict, 20)

In [214]:
with open(path_container + 'id_ingredient_dict.pickle', 'rb') as f:
    id_ingredient_dict = pickle.load(f)

print(id_ingredient_dict[6694])
type(id_ingredient_dict), len(id_ingredient_dict)

Franks Hot Sauce


(dict, 6714)

In [215]:
with open(path_container + 'ingredient_id_dict.pickle', 'rb') as f:
    ingredient_id_dict = pickle.load(f)

print(ingredient_id_dict['lemon grass'])
print(ingredient_id_dict['lemongrass'])
print(ingredient_id_dict['bread'])
type(ingredient_id_dict), len(ingredient_id_dict)

37
561
[698, 1039, 4315]


(dict, 6694)

### Datastet 가져오기 (2) h5py

In [ ]:
"""
_train = [arr_data_train, bin_data_train, int_labels_train, bin_labels_train]
_valid_class = [arr_data_valid_class, bin_data_valid_class, int_labels_valid_class, bin_labels_valid_class]
_valid_compl = [arr_data_valid_compl, bin_data_valid_compl, int_labels_valid_compl, bin_labels_valid_compl]
_test_class = [arr_data_test_class, bin_data_test_class, None, None]
_test_compl = [arr_data_test_compl, bin_data_test_compl, None, None]
"""

In [224]:
with h5py.File(path_container + 'train', 'r') as f:
    arr_data_train = f['arr_data'][:]
    bin_data_train = f['bin_data'][:]
    int_labels_train = f['int_labels'][:]
    bin_labels_train = f['bin_labels'][:]

print(arr_data_train.shape)
print(bin_data_train.shape)
print(int_labels_train.shape)
print(bin_labels_train.shape)

(23547, 65)
(23547, 6714)
(23547,)
(23547, 20)


In [225]:
with h5py.File(path_container + 'valid_class', 'r') as f:
    arr_data_valid_class = f['arr_data'][:]
    bin_data_valid_class = f['bin_data'][:]
    int_labels_valid_class = f['int_labels'][:]
    bin_labels_valid_class = f['bin_labels'][:]

print(arr_data_valid_class.shape)
print(bin_data_valid_class.shape)
print(int_labels_valid_class.shape)
print(bin_labels_valid_class.shape)

(7848, 65)
(7848, 6714)
(7848,)
(7848, 20)


In [226]:
with h5py.File(path_container + 'valid_compl', 'r') as f:
    arr_data_valid_compl = f['arr_data'][:]
    bin_data_valid_compl = f['bin_data'][:]
    int_labels_valid_compl = f['int_labels'][:]
    bin_labels_valid_compl = f['bin_labels'][:]

print(arr_data_valid_compl.shape)
print(bin_data_valid_compl.shape)
print(int_labels_valid_compl.shape)
print(bin_labels_valid_compl.shape)

(7848, 65)
(7848, 6714)
(7848,)
(7848, 6714)


In [227]:
with h5py.File(path_container + 'test_class', 'r') as f:
    arr_data_test_class = f['arr_data'][:]
    bin_data_test_class = f['bin_data'][:]

print(arr_data_test_class.shape)
print(bin_data_test_class.shape)

(3924, 65)
(3924, 6714)


In [228]:
with h5py.File(path_container + 'test_compl', 'r') as f:
    arr_data_test_compl = f['arr_data'][:]
    bin_data_test_compl = f['bin_data'][:]

print(arr_data_test_compl.shape)
print(bin_data_test_compl.shape)

(3924, 65)
(3924, 6714)


In [233]:
# -1로 padding한 것을 6714로 갈아치우기 - for Embedding layer
pad_idx = 6714

arr_data_train[arr_data_train == -1] = pad_idx
arr_data_valid_class[arr_data_valid_class == -1] = pad_idx
arr_data_valid_compl[arr_data_valid_compl == -1] = pad_idx
arr_data_test_class[arr_data_test_class == -1] = pad_idx
arr_data_test_compl[arr_data_test_compl == -1] = pad_idx

### Ingredient feature vectors (nn.Embedding)

In [234]:
arr_data_train[0]

array([2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
       6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714])

In [602]:
num_ingredients = pad_idx +1  # = len(id_ingredient_dict) + 1  # +1 for padding (last)
max_length = 65  # maximum num of ingredients per recipe
embedding_dim = 256


Embed = nn.Embedding(num_embeddings=num_ingredients, embedding_dim=embedding_dim, padding_idx=-1)
x = torch.LongTensor(arr_data_train[:2])

print(Embed.weight)
print(Embed.weight.size())

Parameter containing:
tensor([[-1.5085,  1.3205, -0.5292,  ..., -1.7537, -0.2567, -1.2333],
        [ 0.4654,  0.8227, -1.6501,  ..., -1.0737,  0.7688, -0.6282],
        [ 0.5623, -0.7719,  0.6491,  ...,  1.9821,  0.3590, -1.4031],
        ...,
        [-0.8299,  0.3412, -0.0203,  ..., -0.2970,  0.5006, -0.1194],
        [-0.8445, -0.4461, -0.4429,  ..., -0.7884,  0.5765, -1.4849],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       requires_grad=True)
torch.Size([6715, 256])


In [603]:
with torch.no_grad():
    print(Embed.weight[0])

tensor([-1.5085,  1.3205, -0.5292, -0.8147, -0.2471,  1.6666, -1.7047,  1.5168,
        -0.9718, -2.2025,  0.7877,  1.6499,  0.1197,  1.1889,  1.6560,  2.8253,
         1.6521,  0.9913, -0.4929, -1.4329, -1.5740,  1.7747, -0.6462,  1.5872,
        -0.0715, -0.9060, -0.5537,  2.2482,  0.4627,  1.0123, -0.9640,  0.4060,
         1.2748, -2.3461,  0.1684,  0.0468, -0.7596,  0.7578,  0.0778, -0.4288,
         0.9581, -0.9092,  0.0042,  0.5394, -0.5707, -0.7965, -0.5663,  0.1300,
        -0.7374,  0.0830,  0.8134, -1.0099, -0.5603, -1.0637, -1.3713,  0.8295,
         1.0266,  1.3354,  0.3258, -0.0153,  1.2029,  0.0743,  1.3934, -1.1238,
         1.3378,  0.9338, -0.2577,  0.1934, -0.7783,  0.4122, -0.7936, -1.0152,
         0.3274, -1.5270,  1.9012, -0.0071,  1.3207, -1.5196, -1.0794,  0.3727,
         1.9110,  0.3411,  0.9885, -0.0055,  1.1217,  0.0272,  0.6174, -0.5513,
        -0.1308,  1.1469,  0.5315,  0.0492, -1.4430,  0.3096,  0.1211,  1.5994,
         2.0192, -0.9977,  0.9147,  0.09

In [604]:
print(x)
with torch.no_grad():
    print(Embed(x).size())
    print(Embed(x)[0][11]) # zero padded feature matrix! (0번째 recipe는 9개라 10~65번째 feature vector는 zeros.)

tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
torch.Size([2, 65, 256])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

### Models

Set transformer: ( https://github.com/juho-lee/set_transformer )

In [633]:
## Building blocks of Set Transformers ##
# added masks.

class MAB(nn.Module):
    def __init__(self, dim_Q, dim_K, dim_V, num_heads, ln=False):
        super(MAB, self).__init__()
        self.dim_V = dim_V
        self.num_heads = num_heads
        self.fc_q = nn.Linear(dim_Q, dim_V)
        self.fc_k = nn.Linear(dim_K, dim_V)
        self.fc_v = nn.Linear(dim_K, dim_V)
        if ln:
            self.ln0 = nn.LayerNorm(dim_V)
            self.ln1 = nn.LayerNorm(dim_V)
        self.fc_o = nn.Sequential(
            nn.Linear(dim_V, 2*dim_V),
            nn.ReLU(),
            nn.Linear(2*dim_V, dim_V))

    def forward(self, Q, K, mask=None):
        # Q (batch, q_len, d_hid)
        # K (batch, k_len, d_hid)
        # V (batch, v_len, d_hid == dim_V)
        Q = self.fc_q(Q)
        K, V = self.fc_k(K), self.fc_v(K)
        
        dim_split = self.dim_V // self.num_heads
        
        # Q_ (batch * num_heads, q_len, d_hid // num_heads)
        # K_ (batch * num_heads, k_len, d_hid // num_heads)
        # V_ (batch * num_heads, v_len, d_hid // num_heads)
        Q_ = torch.cat(Q.split(dim_split, 2), 0)
        K_ = torch.cat(K.split(dim_split, 2), 0)
        V_ = torch.cat(V.split(dim_split, 2), 0)
        
        # energy (batch * num_heads, q_len, k_len)
        energy = Q_.bmm(K_.transpose(1,2))/math.sqrt(self.dim_V)
        if mask is not None:
            energy.masked_fill_(mask, float('-inf'))
        A = torch.softmax(energy, 2)
        
        # O (batch, q_len, d_hid)
        O = torch.cat((Q_ + A.bmm(V_)).split(Q.size(0), 0), 2)
        O = O if getattr(self, 'ln0', None) is None else self.ln0(O)
        O = O + self.fc_o(O)
        O = O if getattr(self, 'ln1', None) is None else self.ln1(O)
        return O

class SAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, ln=False):
        super(SAB, self).__init__()
        self.mab = MAB(dim_in, dim_in, dim_out, num_heads, ln=ln)

    def forward(self, X, mask=None):
        return self.mab(X, X, mask=mask)

class ISAB(nn.Module):
    def __init__(self, dim_in, dim_out, num_heads, num_inds, ln=False):
        super(ISAB, self).__init__()
        self.I = nn.Parameter(torch.Tensor(1, num_inds, dim_out))
        nn.init.xavier_uniform_(self.I)
        self.mab0 = MAB(dim_out, dim_in, dim_out, num_heads, ln=ln)
        self.mab1 = MAB(dim_in, dim_out, dim_out, num_heads, ln=ln)

    def forward(self, X, mask=None):
        H = self.mab0(self.I.repeat(X.size(0), 1, 1), X, mask=mask)
        return self.mab1(X, H)

class PMA(nn.Module):
    def __init__(self, dim, num_heads, num_seeds, ln=False):
        super(PMA, self).__init__()
        self.S = nn.Parameter(torch.Tensor(1, num_seeds, dim))
        nn.init.xavier_uniform_(self.S)
        self.mab = MAB(dim, dim, dim, num_heads, ln=ln)
        
    def forward(self, X, mask=None):
        return self.mab(self.S.repeat(X.size(0), 1, 1), X, mask=mask)

In [634]:
class Encoder(nn.Module):
    def __init__(self, dim_input,
                 num_inds=32, dim_hidden=128, num_heads=4, num_layers=2, ln=False):
        super(Encoder, self).__init__()
        self.enc = nn.ModuleList(
            [ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln) for _ in range(num_layers-1)]
        )
        
    def forward(self, X, mask=None):
        out = X.detach()
        for module in self.enc:
            out = module(out, mask=mask)
        return out

class Decoder(nn.Module):
    def __init__(self, num_outputs,
                 num_inds=32, dim_hidden=128, num_heads=4, ln=False):
        super(Decoder, self).__init__()
        self.dec = nn.Sequential(
                PMA(dim_hidden, num_heads, num_outputs, ln=ln),
                SAB(dim_hidden, dim_hidden, num_heads, ln=ln),
                SAB(dim_hidden, dim_hidden, num_heads, ln=ln))
        
    def forward(self, X):
        return self.dec(X)

#### Examples

In [635]:
x = torch.LongTensor(arr_data_train[:2])

with torch.no_grad():
    print('input size', x.size())
    print('input', x)
    feature = Embed(x)  # Size 2, 65, 256
    print('feature size', feature.size())
    print('feature', feature)

input size torch.Size([2, 65])
input tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
feature size torch.Size([2, 65, 256])
feature tensor([[[-0.6592,  0.5575

In [636]:
enc_mask = (x == pad_idx).unsqueeze(1).repeat(4,1,1)
enc_mask.size(), enc_mask

(torch.Size([8, 1, 65]),
 tensor([[[False, False, False, False, False, False, False, False, False,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True]],
 
         [[False, False, False, False, False, False, False, False, False, False,
           False, False,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  True,  True,  True,  True,  True,  True,  True,
            True,  True,  True,  T

In [637]:
x1 = deepcopy(x)
x1[0][:2] = x[0][[1,0]]
print('x1 size', x1.size())
print('x1', x1)
with torch.no_grad():
    feature1 = Embed(x1)
    print('feature1', feature1)

x1 size torch.Size([2, 65])
x1 tensor([[3146, 2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])
feature1 tensor([[[ 2.2880,  0.3431,  1.1873,  ...,  0.9160, -0.3171, -1.3607]

In [638]:
_sab = SAB(256, 128, 4)

In [639]:
with torch.no_grad():
    z = _sab(feature, mask=enc_mask)
    print('sab(feature)',z)
    print(z.size())

sab(feature) tensor([[[ 0.5165,  0.3423, -0.7918,  ...,  0.3747, -0.4377, -0.1863],
         [ 0.1475, -1.1637, -0.1631,  ..., -0.8286, -0.3840, -0.5151],
         [ 0.6952, -0.4090,  0.6754,  ..., -0.3958, -0.4861,  0.0334],
         ...,
         [-0.0704, -0.2253, -0.2671,  ..., -0.0589, -0.0949, -0.1920],
         [-0.0704, -0.2253, -0.2671,  ..., -0.0589, -0.0949, -0.1920],
         [-0.0704, -0.2253, -0.2671,  ..., -0.0589, -0.0949, -0.1920]],

        [[-0.0725,  0.1918,  0.5322,  ..., -0.2370, -1.3910,  0.1049],
         [-0.4460, -0.8585,  0.2142,  ..., -0.0111, -1.0845,  0.3749],
         [-1.1908, -0.5597,  1.1593,  ...,  0.9088,  0.3672, -0.1515],
         ...,
         [ 0.0911, -0.1334,  0.0170,  ..., -0.0028, -0.1610,  0.1357],
         [ 0.0911, -0.1334,  0.0170,  ..., -0.0028, -0.1610,  0.1357],
         [ 0.0911, -0.1334,  0.0170,  ..., -0.0028, -0.1610,  0.1357]]])
torch.Size([2, 65, 128])


In [640]:
_enc = Encoder(256, dim_hidden=128) 

In [641]:
with torch.no_grad():
    code = _enc(feature,mask=enc_mask)
    print(feature.size())
    print(code[0][15:])
    print(code.size())  # Size 2, 65, dim_hidden

torch.Size([2, 65, 256])
tensor([[ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
        [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
        [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
        ...,
        [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
        [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
        [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174]])
torch.Size([2, 65, 128])


In [642]:
with torch.no_grad():
    code1 = _enc(feature1, mask=enc_mask)
    print(code1)
    print(code1.size())  # Size 2, 65, dim_hidden  # permutation equivariant! (observe first 2 rows)

tensor([[[-0.3653, -0.2419,  0.2929,  ...,  0.2913, -0.6550,  0.6975],
         [-0.2268, -0.2857, -0.2599,  ...,  0.4555,  0.0017,  0.2129],
         [ 0.1202, -0.3273,  0.1191,  ..., -0.6078, -0.1978,  0.2386],
         ...,
         [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
         [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174],
         [ 0.0304, -0.0662, -0.0070,  ..., -0.0596, -0.2768,  0.2174]],

        [[-0.2721, -0.4374,  0.3580,  ..., -0.3973, -0.2365, -0.1932],
         [-0.3750, -0.1942,  0.2701,  ..., -0.1424,  0.4709,  0.1798],
         [-0.1730, -0.0925, -0.1718,  ..., -0.2075,  1.1529,  0.8532],
         ...,
         [-0.2592, -0.1330, -0.0783,  ..., -0.1548,  0.0371,  0.1877],
         [-0.2592, -0.1330, -0.0783,  ..., -0.1548,  0.0371,  0.1877],
         [-0.2592, -0.1330, -0.0783,  ..., -0.1548,  0.0371,  0.1877]]])
torch.Size([2, 65, 128])


In [643]:
_dec = Decoder(1, dim_hidden=128)

In [644]:
with torch.no_grad():
    out = _dec(code)
    print(out)
    print(out.size())  # Size 2, num_output, dim_hidden. FF 적용 직전 vector.

tensor([[[-0.1508, -0.2508, -0.1915,  0.3444, -0.2414, -0.0401,  0.1389,
           0.0513, -0.1571,  0.2136,  0.0504, -0.0411,  0.0559,  0.0982,
           0.1302, -0.0359,  0.1272, -0.2365,  0.3331,  0.1819,  0.2597,
          -0.0376,  0.1688,  0.1096, -0.1450,  0.0516, -0.0742,  0.0605,
           0.0728, -0.0476,  0.1194, -0.0662, -0.1504,  0.0394,  0.0018,
          -0.3478,  0.1906, -0.0150,  0.3143, -0.3142, -0.1716, -0.1427,
           0.3266, -0.0731,  0.0844,  0.0579, -0.0226,  0.0569, -0.0264,
           0.1366, -0.0414, -0.0115, -0.1819, -0.1998, -0.0806, -0.2261,
          -0.1732, -0.1733,  0.1483,  0.0928, -0.0470, -0.2604, -0.0687,
          -0.0596, -0.1244, -0.0706,  0.1617,  0.3518, -0.0511,  0.1004,
           0.0192,  0.1413, -0.0120,  0.2418, -0.0773,  0.4252,  0.0875,
           0.1626,  0.3770, -0.1370,  0.0737, -0.0687,  0.1294,  0.1120,
           0.0639,  0.0577,  0.0919, -0.2514, -0.1257, -0.1431, -0.0204,
           0.1035, -0.2529,  0.3842,  0.0557, -0.06

In [645]:
# permutation invariance

with torch.no_grad():
    x2 = deepcopy(x)
    """3146, 2813, 3229, 3885, 4379, 4390, 5250, 5456, 6187"""
    shuf_ind = torch.randperm(x2.size(1))
    x2[0] = x2[0][shuf_ind]  # ramdom permuted!
    print(x2[0])
    feature2 = Embed(x2)
    enc_mask2 = deepcopy(enc_mask)
    enc_mask2[0] = enc_mask[0][0][shuf_ind]
    code2 = _enc(feature2, mask=enc_mask2)
    out2 = _dec(code2)
    print(out2.size())
    print(out2[0].isclose(out[0]))
    print(out2[0] - out[0]) # almost the same.

tensor([6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 3146, 6714, 6714, 6714,
        6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        3229, 6714, 4379, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
        6714, 6714, 6714, 6714, 6714, 4390, 6714, 6714, 6714, 3885, 6714, 6714,
        6714, 6187, 6714, 5250, 6714, 6714, 6714, 2813, 6714, 6714, 6714, 5456,
        6714, 6714, 6714, 6714, 6714])
torch.Size([2, 1, 128])
tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False

### Unified Model: Classification + Completion (CCNet)

In [695]:
x

tensor([[2813, 3146, 3229, 3885, 4379, 4390, 5250, 5456, 6187, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714],
        [ 392,  937, 1476, 2172, 2351, 2813, 3350, 3554, 3857, 3978, 5249, 5648,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714, 6714,
         6714, 6714, 6714, 6714, 6714]])

In [750]:
def make_one_hot(x):
    if type(x) is not torch.Tensor:
        x = torch.LongTensor(x)
    if x.dim() > 2:
        x = x.squeeze()
        if x.dim() > 2:
            return False
    elif x.dim() < 2:
        x = x.unsqueeze(0)
    return F.one_hot(x).sum(1)[:,:-1]

In [689]:
make_one_hot(x).size()

torch.Size([2, 6714])

In [769]:
class CCNet(nn.Module):
    def __init__(self, dim_input=256,
                 dim_output=20,
                 num_items=6714+1, 
                 num_inds=32, 
                 dim_hidden=128, 
                 num_heads=4, 
                 num_outputs=1+1,  # class 1 + compl 1
                 num_enc_layers=4, 
                 num_dec_layers=2, 
                 ln=False, 
                 classify=True, 
                 complete=True):
        super(CCNet, self).__init__()
        
        self.num_heads = num_heads
        self.padding_idx = num_items-1
        self.classify, self.complete = classify, complete
        self.embedding =  nn.Embedding(num_embeddings=num_items, embedding_dim=dim_input, padding_idx=-1)
        self.encoder = nn.ModuleList(
            [ISAB(dim_input, dim_hidden, num_heads, num_inds, ln=ln)] +
            [ISAB(dim_hidden, dim_hidden, num_heads, num_inds, ln=ln) for _ in range(num_enc_layers-1)])
        self.pooling = PMA(dim_hidden, num_heads, num_outputs, ln=ln)
        self.decoder1 = nn.Sequential(
            *[SAB(dim_hidden, dim_hidden, num_heads, ln=ln) for _ in range(num_dec_layers)])
        self.ff1 = nn.Sequential(
            nn.Linear(dim_hidden, dim_hidden),
            nn.ReLU(),
            nn.Linear(dim_hidden, dim_output))
        self.decoder2 = nn.ModuleList(
            [MAB(dim_hidden, dim_input, dim_hidden, num_heads, ln=ln) for _ in range(2)])
        self.ff2 = nn.Linear(dim_hidden, num_items-1)

    
    def forward(self, x):
        # x(=recipes): (batch, max_num_ingredient=65)
        # bin_x : (batch, num_items-1=6714)
        if not (self.classify or self.complete):
            return
        
        x = torch.LongTensor(x)
        feature = self.embedding(x)
        # feature: (batch, max_num_ingredient=65, dim_input=256)
        # cf. embedding.weight: (num_items=6715, dim_input=256)
        mask = (x == self.padding_idx).repeat(self.num_heads,1).unsqueeze(1)
        # mask: (batch*num_heads, 1, max_num_ingredient=65)
        code = feature.clone()
        for module in self.encoder:
            code = module(code, mask=mask)
        # code: (batch, max_num_ingredient=65, dim_hidden=128) : permutation-equivariant.
        
        pooled = self.pooling(code, mask=mask)
        # pooled: (batch, num_outputs=2, dim_hidden=128) : permutation-invariant.
        
        signals = self.decoder1(pooled)
        # no mask; signals: (batch, num_outputs=2, dim_hidden=128) : permutation-invariant.
        
        # split two signals: for classification & completion.
        signal_classification = signals[:][0]                   # (batch, dim_hidden=128)
        signal_completion = signals.clone()[:][1].unsqueeze(1)  # (batch, 1, dim_hidden=128)
        
        # Classification:
        if self.classify:
            logit_classification = self.ff1(signal_classification)  # (batch, dim_output)
            if not self.complete:
                return logit_clasification
        
        # Completion:
        if self.complete:
            bool_x = (make_one_hot(x) == True)
            used_ingred_mask = bool_x.repeat(self.num_heads,1).unsqueeze(1)
            # used_ingred_mask: (batch*num_heads, 1, num_items-1=6714)
            
            embedding_weight = self.embedding.weight[:-1].unsqueeze(0).repeat(feature.size(0),1,1)
            # embedding_weight: (batch, num_items=6715, dim_input=256)
            
            for module in self.decoder2:
                signal_completion = module(signal_completion, embedding_weight, mask=used_ingred_mask)
            logit_completion = self.ff2(signal_completion.squeeze()) # (batch, num_items-1=6714)
            logit_completion[bool_x] = float('-inf')
            if not self.classify:
                return logit_completion

        return logit_classification, logit_completion

In [770]:
_model = CCNet()

In [771]:
with torch.no_grad():
    logit1, logit2 = _model(arr_data_train[:2])
    print(logit1, logit1.size())
    print(logit2, logit2.size())
    

tensor([[ 0.0206, -0.0218, -0.0604,  0.0219, -0.0341,  0.0685,  0.0029, -0.0262,
          0.0789, -0.0135, -0.0330, -0.0573,  0.0831, -0.0547,  0.0514,  0.0190,
         -0.0397,  0.0662, -0.0025,  0.0149],
        [ 0.0148, -0.0124, -0.0635,  0.0235, -0.0381,  0.0614,  0.0021, -0.0336,
          0.0783, -0.0071, -0.0352, -0.0504,  0.0816, -0.0599,  0.0446,  0.0217,
         -0.0373,  0.0713, -0.0068,  0.0105]]) torch.Size([2, 20])
tensor([[-0.0829,  0.0355,  0.0451,  ...,  0.0636, -0.0870, -0.1389],
        [-0.0709,  0.0328,  0.0472,  ...,  0.0696, -0.0889, -0.1402]]) torch.Size([2, 6714])


In [772]:
# Sanity check
x = torch.LongTensor(arr_data_train[:2])
bin_x = make_one_hot(x)

print((F.softmax(logit2, dim=-1) == 0).any())
logit2[0][torch.where(bin_x[0]==1)[0]]

tensor(True)


tensor([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf])